In [168]:
import os
import csv
import json
import pandas as pd
import datetime
from google.cloud import storage
from supabase import create_client

In [54]:
# connect to gcloud bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'survai-data-connect.json'
storage_client = storage.Client()
bucket_name = 'survai-dataset'
bucket = storage_client.bucket(bucket_name)

# Connect to database
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
email = os.environ.get("EMAIL")
password = os.environ.get("PASSWORD")
client = create_client(url, key)
user = client.auth.sign_in(email=email, password=password)

In [56]:
table = 'patriot_front_data'

In [182]:
# access temp_videodata_storage folder
blobs = storage_client.list_blobs(bucket_name, prefix='datasets/test_datasets/sparverius_pf/results_08052022/temp_videodata_storage/')
file_list = []

for blob in blobs:

    file = blob.name.split('/')[5]

    # filter for desired files
    if 'od.json' in file or 'stats.json' in file:
        i = file.count('_') # count instances of '_' in file name

        if i > 1:
            file_name = "_".join(file.split("_", i)[:i]) # split on last '_'
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': f'{file_name}.mp4'}).execute()

        else:
            file_name = file.split('_')[0]
            if file_name not in file_list:
                file_list.append(file_name)
                client.table(table).insert({'video': f'{file_name}.mp4'}).execute()

            
    if 'od.json' in file:
        od = json.loads(blob.download_as_string())

        # change seconds to datetime
        for k,v in list(od.items()):
            newkey = str(datetime.timedelta(seconds=int(k)))
            od[newkey] = od.pop(k)

        client.table(table).update({'detections_by_second': od}).eq('video', f'{file_name}.mp4').execute()

    if 'stats.json' in file:
        stats = json.loads(blob.download_as_string())
        for k,v in stats.items():
            client.table(table).update({k: v}).eq('video', f'{file_name}.mp4').execute()


In [183]:
df = pd.read_csv('patriot_front_data.csv')
print(df)

,video,Baton,Pepper Spray,Uniformed,Chemical Smoke,Gun,Riot Shield,Non Uniformed,spray,running,crowd,brawling,person_on_ground,throwing,arresting,striking,violent_actions,detections_by_second
0,009.mp4,NaN,NaN,NaN,NaN,10.0,13.0,150.0,25.0,NaN,NaN,NaN,33.0,NaN,NaN,NaN,0.0,[object Object]
1,2021-11-14_01.13.50.mp4,NaN,13.0,NaN,NaN,NaN,NaN,47.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[object Object]
2,1_4934073479210205678.mp4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[object Object]
3,008.mp4,NaN,NaN,6.0,NaN,NaN,1.0,40.0,2.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,0.0,[object Object]
4,2021-11-20_11.20.14.mp4,NaN,NaN,NaN,NaN,NaN,NaN,20.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[object Object]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,WA_DSCF3252.mp4,6.0,NaN,5.0,NaN,16.0,NaN,196.0,40.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.0,[object Object]
425,drilling_draft1.mp4,NaN,NaN,130.0,NaN,10.0,428.0,2738.0,303.0,20.0,136.0,NaN,115.0,NaN,4.0,115.0,119.0,[object Object]
426,latesT_PAtriotfront.mp4,5.0,1.0,335.0,21.0,10.0,86.0,850.0,31.0,NaN,137.0,133.0,296.0,NaN,NaN,NaN,133.0,[object Object]
427,paint_test_20211208_212840.mp4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[object Object]
